In [0]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


the data location on drive


In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/project/SD_products.csv')

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137330 entries, 0 to 137329
Data columns (total 9 columns):
_id                137330 non-null object
category           137330 non-null object
details            136844 non-null object
discountedPrice    87333 non-null float64
imageUrl           137297 non-null object
name               137000 non-null object
originalPrice      136964 non-null float64
subCategory        137330 non-null object
subSubCategory     137318 non-null object
dtypes: float64(2), object(7)
memory usage: 9.4+ MB


In [0]:
data = data[['category', 'details', 'name', 'subCategory', 'subSubCategory']]

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137330 entries, 0 to 137329
Data columns (total 5 columns):
category          137330 non-null object
details           136844 non-null object
name              137000 non-null object
subCategory       137330 non-null object
subSubCategory    137318 non-null object
dtypes: object(5)
memory usage: 5.2+ MB


In [0]:
data = data.dropna()

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136832 entries, 0 to 137329
Data columns (total 5 columns):
category          136832 non-null object
details           136832 non-null object
name              136832 non-null object
subCategory       136832 non-null object
subSubCategory    136832 non-null object
dtypes: object(5)
memory usage: 6.3+ MB


In [0]:
data['desc'] = data[['name', 'details']].apply(lambda x: ' '.join(x), axis=1)

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136832 entries, 0 to 137329
Data columns (total 6 columns):
category          136832 non-null object
details           136832 non-null object
name              136832 non-null object
subCategory       136832 non-null object
subSubCategory    136832 non-null object
desc              136832 non-null object
dtypes: object(6)
memory usage: 7.3+ MB


In [0]:
data = data.drop_duplicates()

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108464 entries, 0 to 137329
Data columns (total 6 columns):
category          108464 non-null object
details           108464 non-null object
name              108464 non-null object
subCategory       108464 non-null object
subSubCategory    108464 non-null object
desc              108464 non-null object
dtypes: object(6)
memory usage: 5.8+ MB


In [0]:
data['desc'][0]

'Specially Designed Red Rayon Kurti with Jaipuri Jacket Designers Piece'

In [0]:
import re 
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
def data_preprocessing(X):
  documents = []
  # X = data1['details']

  stemmer = WordNetLemmatizer()

  for i in X:  
      
      document = str(i)
      # Remove all the special characters
      document = re.sub(r'\W', ' ', document) 

      # remove all single characters
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

      #remove all the no
      address = re.sub(r'^[\d-]+ ', '', document)

      # Remove single characters from the start
      document = re.sub(r'\^[a-zA-Z]\s+', '', document) 

      # Substituting multiple spaces with single space
      document = re.sub(r'\s+', ' ', document, flags=re.I)
      
      document = document.strip(' ')


      # Converting to Lowercase
      document = document.lower()

      # Lemmatization
      document = document.split()

      document = [stemmer.lemmatize(word) for word in document]
      document = ' '.join(document)

      documents.append(document)
      
  return documents 

In [0]:
X = data_preprocessing(data['desc'])

In [0]:
X[0]

'specially designed red rayon kurti with jaipuri jacket designer piece'

In [0]:
data1 = data[['category', 'desc']]

In [0]:
data1.shape

(108464, 2)

In [0]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108464 entries, 0 to 137329
Data columns (total 2 columns):
category    108464 non-null object
desc        108464 non-null object
dtypes: object(2)
memory usage: 7.5+ MB


In [0]:
data1 = data1.drop_duplicates()

In [0]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13076 entries, 0 to 137188
Data columns (total 2 columns):
category    13076 non-null object
desc        13076 non-null object
dtypes: object(2)
memory usage: 306.5+ KB


In [0]:
x_data = data_preprocessing(data1['desc'])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
text_tf= tf.fit_transform(x_data)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_tf, data1['category'], test_size=0.3, random_state=1)

In [0]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Accuracy: 0.7593730792870313


In [0]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predicted)) 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                      precision    recall  f1-score   support

          Activities       0.25      0.04      0.06      1232
                Auto       0.00      0.00      0.00      1150
         Electronics       0.96      0.76      0.85      8756
Grocery and Crockery       0.00      0.00      0.00       604
  Health and Fitness       0.00      0.00      0.00       677
     Home and Living       0.66      0.99      0.79     12989
                Kids       0.78      0.89      0.83      2152
                 Men       0.96      0.75      0.84       141
        Pet Supplies       0.00      0.00      0.00        59
           Sastobook       0.95      0.62      0.75      4324
               Women       0.94      0.85      0.89       456

            accuracy                           0.76     32540
           macro avg       0.50      0.45      0.46     32540
        weighted avg       0.73      0.76      0.72     32540



In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1).fit(X_train, y_train)
predicted= clf.predict(X_test)
print("logistic regression Accuracy:",metrics.accuracy_score(y_test, predicted))

logistic regression Accuracy: 0.7741241548862938


In [0]:
data2 = data[['desc','subCategory']]

In [0]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108464 entries, 0 to 137329
Data columns (total 2 columns):
desc           108464 non-null object
subCategory    108464 non-null object
dtypes: object(2)
memory usage: 7.5+ MB


In [0]:
data2 = data2.drop_duplicates()

In [0]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26598 entries, 0 to 137188
Data columns (total 2 columns):
desc           26598 non-null object
subCategory    26598 non-null object
dtypes: object(2)
memory usage: 623.4+ KB


In [0]:
data2['subCategory'].value_counts()

Genres                           2503
Home Appliances                  1733
Home Decor                       1494
Kitchen and Dining               1324
Clothing                         1178
Cleaning and Toiletries          1084
Fitness                          1037
Accessories                       942
Celebration                       903
Home Safety                       896
Tools                             867
Groceries                         799
Body and Frames                   765
Computer Accessories              751
Home Organization                 713
Health Care                       707
Lighting                          700
Gardening                         695
Toys                              682
Auto Care                         657
Rider's Accessories               637
Adventure Sports                  631
Exciting Eletronics               478
Audio Video                       473
Alarm and Security System         444
Baby Care                         432
Camera and A

In [0]:
data2 = data2.groupby('subCategory').filter(lambda x: len(x) >= 100)


In [0]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26502 entries, 0 to 137188
Data columns (total 2 columns):
desc           26502 non-null object
subCategory    26502 non-null object
dtypes: object(2)
memory usage: 621.1+ KB


In [0]:
X = data_preprocessing(data2['desc'])

In [0]:
tf=TfidfVectorizer()
text_tf= tf.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(text_tf, data2['subCategory'], test_size=0.3, random_state=12)

In [0]:
clf = OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1).fit(X_train, y_train)
predicted= clf.predict(X_test)
print("logistic regression Accuracy:",metrics.accuracy_score(y_test, predicted))

logistic regression Accuracy: 0.17897119859137214
